# Titanic - Data Cleaning and Exploration

## Import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

raw_train=pd.read_csv('train.csv')
raw_test=pd.read_csv('test.csv')

train=raw_train
test=raw_test

## Inspection

In [ ]:
#print training dataset
train

In [ ]:
#Get Summary Stats
train.describe()

In [ ]:
#Get information about the Dataframe
train.info()

## Cleaning

In [ ]:
train[train.duplicated()]

In [ ]:
#Find number of Unique values in each column
train.nunique()

In [ ]:
#Looking for Missing Data
train.isnull().sum()

In [ ]:
#Check the two rows with missing Embarked values
train[pd.isnull(train['Embarked'])]

In [ ]:
#We assign the modal value to the missing Embarked data (the [0] after mode because mode is a pandas series)
mode=train['Embarked'].mode()
train['Embarked'].fillna(mode[0],inplace=True)

In [ ]:
#Show the replaced values
train[train['PassengerId'].isin([62,830])]

In [ ]:
#Looking at the Correlation Matrix below, it seems Age is correlated with Pclass
train.corr()

#You could also encode other categorical variables to seem them in the matrix

In [ ]:
#SibSp and Parch are also correlated with Age.
#We could use Regression with Age as target and Pclass, Parch and SibSp as Features.
#https://towardsdatascience.com/predict-missing-values-in-the-dataset-897912a54b7b

#But for now we will simple assign the mean age for the Pclass of the missing data rows

In [ ]:
#Average age different for different Pclass
train.groupby('Pclass').mean()

In [ ]:
#Replace Missing Vales of Age with Mean by Pclass
train['Age'] = train['Age'].fillna(train.groupby(['Pclass'])['Age'].transform('mean'))


#for test data
test['Age'] = test['Age'].fillna(train.groupby(['Pclass'])['Age'].transform('mean'))


In [ ]:
test.isnull().sum()

In [ ]:
#Replace Missing Values for Fare in test Data by Mean of Fare in train data
test['Fare'] = test['Fare'].fillna(train['Fare'].mean())

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
#Design New Feature
train['Cabin_Class'] = train['Cabin'].str[0]

#for test
test['Cabin_Class'] = test['Cabin'].str[0]

In [ ]:
train

In [ ]:
pd.crosstab(train['Cabin_Class'], train['Pclass'], margins = False)

In [ ]:
#Average Fares per Cabin Class
train.groupby(['Cabin_Class']).mean()

In [ ]:
#Replace the Missing values in Cabin_Class with "U" for Unknown
train['Cabin_Class'].fillna('U',inplace=True)


#for test
test['Cabin_Class'].fillna('U',inplace=True)

### One Hot Ecoding Emabrked and Cabin_Class 

In [ ]:
#Splitting Y from X before One Hot Encoding to ensure similar Encoding Dictionary and column numbers for train and test

y_train = train['Survived']
y_train


In [ ]:
train.drop(columns=['Survived'], inplace=True)
train

In [ ]:
from feature_engine.encoding import OneHotEncoder

#Create Instance and Fit
# drop_lastto return k-1, false to return k
ohe = OneHotEncoder(top_categories=None, variables=['Cabin_Class', 'Embarked'], drop_last=True)
ohe.fit(train)

#for test
ohe.fit(test)

In [ ]:
#Transform
temp = ohe.transform(train)
train=temp
train

In [ ]:
#for test
temp2 = ohe.transform(test)

test=temp2
test

In [ ]:
train.isnull().sum()

In [ ]:
#Cabin Feature does not carry much information since many Distinct values and majority values missing.
#Therefore, we drop it along with the other uninformative columns.

train.drop(columns=['Cabin', 'Ticket', 'Name', 'PassengerId'], inplace=True)
train

In [ ]:
#For test
test.drop(columns=['Cabin', 'Ticket', 'Name', 'PassengerId'], inplace=True)
test

### Check for Multicollinearity

In [ ]:
corr = train.corr().abs()

s = corr.unstack()
s

In [ ]:
so = pd.DataFrame(s.sort_values(kind="quicksort",ascending=False))
so[so[0]!=1].head(10)

In [ ]:
#The top 4 might be something to worry about but makes sense that they are the hot encoded features

### Outliers

In [ ]:
#Scatter Plot of Age and Fare to check if older people buy expensive tickets

plt.scatter(train['Age'], train['Fare'])
plt.xlabel('Age')
plt.ylabel('Fare')
plt.show()

In [ ]:
#Show the outliers in Fare
train[train['Fare']>300]

In [ ]:
#We could replace these outliers with the mean if we thought they were erroneous but we ignore them for now.

## Preparing Data for ML

### Encoding Categorical Variables

In [ ]:
#Embarked and Cabin_Class already One Hot Encoded

In [ ]:
#Sex is binary - so Ordinal Encoding
# Using Feature Engine because it allows us to code multiple variables at a time (sklearn doesn't) and allows us to 
#view the dictionary later (pandas doesn't)

from feature_engine.encoding import OrdinalEncoder

In [ ]:
ordinal_enc = OrdinalEncoder(encoding_method='arbitrary', variables=['Sex'])

ordinal_enc.fit(train)

In [ ]:
# in the encoder dict we can observe the numbers
# assigned to each category for all the indicated variables

ordinal_enc.encoder_dict_

In [ ]:
# this is the list of variables that the encoder will transform

ordinal_enc.variables_

In [ ]:
#Transform Train
train = ordinal_enc.transform(train)

train

In [ ]:
#Transform Test using same Encoder object

test = ordinal_enc.transform(test)

test

## Logistic Regression

In [ ]:
#Convert to nparrays
nptrain=train.values
nptest=test.values
y_train=y_train.values

In [ ]:
#Import Class and create Instance
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter = 1000)

In [ ]:
model.fit(nptrain,y_train)

In [ ]:
prediction=pd.DataFrame(model.predict(nptest))

In [ ]:
prediction

### Formatting Results for Submission

In [ ]:
result = pd.read_csv('test.csv')

In [ ]:
result=result['PassengerId']

In [ ]:
result=pd.DataFrame(result)
result

In [ ]:
result['Survived']=prediction

In [ ]:
result

In [ ]:
#Make sure to select index=False when saving
pd.DataFrame(result).to_csv(r".\csv\1.submission.csv",index=False)